<a href="https://colab.research.google.com/github/dpak8842/Generative-AI/blob/main/Vector_search_gemini_elastic_102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install -q -U google-generativeai elasticsearch


Import packages and credentials



In [19]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
from elasticsearch import Elasticsearch, helpers
from getpass import getpass

GOOGLE_API_KEY = getpass("Google API Key :")
ELASTIC_API_KEY = getpass("Elastic API Key :")
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID :")
elastic_index_name = "gemini-demo"

Google API Key :··········
Elastic API Key :··········
Elastic Cloud ID :··········


Embedding generation

In [20]:
genai.configure(api_key=GOOGLE_API_KEY)

title = "Climate in India"
sample_text = "India generally experiences a hot summer from March to June, with temperatures often exceeding 40°C in central and northern regions. Monsoon season, from June to September, brings heavy rainfall, especially in the western coast and northeastern areas. Post-monsoon months, October and November, mark a transition with decreasing rainfall. Winter, from December to February, varies in temperature across the country, with colder conditions in the north and milder weather in the south. India's diverse climate is influenced by its geographical features, resulting in regional "

model = "models/embedding-001"
embedding = genai.embed_content(
    model=model, content=sample_text, task_type="retrieval_document", title=title
)

Connecting Elasticsearch

In [21]:
es = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY)


Index document with Elasticsearch

In [22]:
doc = {"text": sample_text, "text_embedding": embedding["embedding"]}

resp = es.index(index=elastic_index_name, document=doc)

print(resp)

{'_index': 'gemini-demo', '_id': 'pPZyeo4BG0JaeEuGGhLO', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


Searching for document with Elasticsearch

In [23]:
q = "How's weather in India?"

embedding = genai.embed_content(model=model, content=q, task_type="retrieval_query")

resp = es.search(
    index=elastic_index_name,
    knn={
        "field": "text_embedding",
        "query_vector": embedding["embedding"],
        "k": 10,
        "num_candidates": 100,
    },
)


for result in resp["hits"]["hits"]:
    pretty_output = f"\n\nID: {result['_id']}\n\nText: {result['_source']['text']}\n\nEmbedding: {result['_source']['text_embedding']}"
    print(pretty_output)



ID: pPZyeo4BG0JaeEuGGhLO

Text: India generally experiences a hot summer from March to June, with temperatures often exceeding 40°C in central and northern regions. Monsoon season, from June to September, brings heavy rainfall, especially in the western coast and northeastern areas. Post-monsoon months, October and November, mark a transition with decreasing rainfall. Winter, from December to February, varies in temperature across the country, with colder conditions in the north and milder weather in the south. India's diverse climate is influenced by its geographical features, resulting in regional 

Embedding: [0.02232265, -0.011462996, -0.048808888, 0.040615793, 0.04164961, 0.025343282, 0.030588645, -0.021058632, 0.017425675, 0.061474845, -0.026729703, -0.00023359573, 0.0019051981, -0.0067917127, -0.06262906, 0.042496722, -0.0182341, 0.029211545, 0.0031950264, -0.023711342, 0.011959027, 0.016201291, -0.00877714, -0.008762236, 0.005813534, -0.037433643, -0.056903463, -0.042289805, 